In [ ]:
import sys
import time
import gc
import os
import json

import pandas as pd
import numpy as np
import warnings
import pickle
import polars as pl

from collections import defaultdict
from itertools import combinations
import pyarrow as pa

from xgboost import XGBClassifier
from catboost import CatBoostClassifier

from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, f1_score


import matplotlib.pyplot as plt
from colorama import Fore, Back, Style

import warnings
warnings.filterwarnings('ignore')

# CatBoost Part

In [ ]:
dtypes = {"session_id": 'int64',
          "index": np.int16,
          "elapsed_time": np.int32,
          "event_name": 'category',
          "name": 'category',
          "level": np.int8,
          "page": np.float16,
          "room_coor_x": np.float16,
          "room_coor_y": np.float16,
          "screen_coor_x": np.float16,
          "screen_coor_y": np.float16,
          "hover_duration": np.float32,
          "text": 'category',
          "fqid": 'category',
          "room_fqid": 'category',
          "text_fqid": 'category',
          "fullscreen": np.int8,
          "hq": np.int8,
          "music": np.int8,
          "level_group": 'category'
          }
use_col = ['session_id', 'index', 'elapsed_time', 'event_name', 'name', 'level', 'page',
           'room_coor_x', 'room_coor_y', 'hover_duration', 'text', 'fqid', 'room_fqid', 'text_fqid', 'level_group']

targets = pd.read_csv('/kaggle/input/predict-student-performance-from-game-play/train_labels.csv')
targets['session'] = targets.session_id.apply(lambda x: int(x.split('_')[0]) )
targets['q'] = targets.session_id.apply(lambda x: int(x.split('_')[-1][1:]) )

feature_df = pd.read_csv('/kaggle/input/feature-for-cat/feature_sort.csv')

models = {}

list_kol_f = {1: 140, 3: 110, 4: 120, 5: 220, 6: 130, 7: 110, 8: 110, 9: 100, 10: 140, 11: 120, 14: 160, 15: 160, 16: 130, 17: 140}

In [ ]:
def delt_time_def(df):
    df.sort_values(by=['session_id', 'elapsed_time'], inplace=True)
    df['d_time'] = df['elapsed_time'].diff(1)
    df['d_time'].fillna(0, inplace=True)
    df['delt_time'] = df['d_time'].clip(0, 103000)
    df['delt_time_next'] = df['delt_time'].shift(-1)
    return df


def feature_engineer(train, kol_f):
    global kol_col, kol_col_max
    kol_col = 9
    kol_col_max = 11+kol_f*2
    col = [i for i in range(0,kol_col_max)]
    new_train = pd.DataFrame(index=train['session_id'].unique(), columns=col, dtype=np.float16)  
    new_train[10] = new_train.index # "session_id"    

    new_train[0] = train.groupby(['session_id'])['d_time'].quantile(q=0.3)
    new_train[1] = train.groupby(['session_id'])['d_time'].quantile(q=0.8)
    new_train[2] = train.groupby(['session_id'])['d_time'].quantile(q=0.5)
    new_train[3] = train.groupby(['session_id'])['d_time'].quantile(q=0.65)
    new_train[4] = train.groupby(['session_id'])['hover_duration'].agg('mean')
    new_train[5] = train.groupby(['session_id'])['hover_duration'].agg('std')    
    new_train[6] = new_train[10].apply(lambda x: int(str(x)[:2])).astype(np.uint8) # "year"
    new_train[7] = new_train[10].apply(lambda x: int(str(x)[2:4])+1).astype(np.uint8) # "month"
    new_train[8] = new_train[10].apply(lambda x: int(str(x)[4:6])).astype(np.uint8) # "day"
    new_train[9] = new_train[10].apply(lambda x: int(str(x)[6:8])).astype(np.uint8) + new_train[10].apply(lambda x: int(str(x)[8:10])).astype(np.uint8)/60
    new_train[10] = 0
    new_train = new_train.fillna(-1)
    
    return new_train


def feature_next_t(row_f, new_train, train, gran_1, gran_2, i):
    global kol_col
    kol_col +=1
    col1 = row_f['col1']
    val1 = row_f['val1']
    maska = (train[col1] == val1)
    if row_f['kol_col'] == 1:       
        new_train[kol_col] = train[maska].groupby(['session_id'])['delt_time_next'].sum()
        if gran_1:
            kol_col +=1
            new_train[kol_col] = train[maska].groupby(['session_id'])['delt_time'].mean()
        if gran_2:
            kol_col +=1
            new_train[kol_col] = train[maska].groupby(['session_id'])['index'].count()          
    elif row_f['kol_col'] == 2: 
        col2 = row_f['col2']
        val2 = row_f['val2']
        maska = maska & (train[col2] == val2)        
        new_train[kol_col] = train[maska].groupby(['session_id'])['delt_time_next'].sum()
        if gran_1:
            kol_col +=1
            new_train[kol_col] = train[maska].groupby(['session_id'])['delt_time'].mean()
        if gran_2:
            kol_col +=1
            new_train[kol_col] = train[maska].groupby(['session_id'])['index'].count()
    return new_train


def feature_next_t_otvet(row_f, new_train, train, gran_1, gran_2, i):
    global kol_col
    kol_col +=1
    col1 = row_f['col1']
    val1 = row_f['val1']
    maska = (train[col1] == val1)
    if row_f['kol_col'] == 1:      
        new_train[kol_col] = train[maska]['delt_time_next'].sum()
        if gran_1:
            kol_col +=1
            new_train[kol_col] = train[maska]['delt_time'].mean()
        if gran_2:
            kol_col +=1
            new_train[kol_col] = train[maska]['index'].count()          
    elif row_f['kol_col'] == 2: 
        col2 = row_f['col2']
        val2 = row_f['val2']
        maska = maska & (train[col2] == val2)        
        new_train[kol_col] = train[maska]['delt_time_next'].sum()
        if gran_1:
            kol_col +=1
            new_train[kol_col] = train[maska]['delt_time'].mean()
        if gran_2:
            kol_col +=1
            new_train[kol_col] = train[maska]['index'].count()
    return new_train


def experiment_feature_next_t_otvet(row_f, new_train, train, gran_1, gran_2, i):
    global kol_col
    kol_col +=1
    if row_f['kol_col'] == 1: 
        maska = train[row_f['col1']] == row_f['val1']
        new_train[kol_col] = train[maska]['delt_time_next'].sum()
        if gran_1:
            kol_col +=1
            new_train[kol_col] = train[maska]['delt_time'].mean()
        if gran_2:
            kol_col +=1
            new_train[kol_col] = train[maska]['index'].count()          
    elif row_f['kol_col'] == 2: 
        col2 = row_f['col2']
        val2 = row_f['val2']
        maska = (train[col1] == val1) & (train[col2] == val2)        
        new_train[kol_col] = train[maska]['delt_time_next'].sum()
        if gran_1:
            kol_col +=1
            new_train[kol_col] = train[maska]['delt_time'].mean()
        if gran_2:
            kol_col +=1
            new_train[kol_col] = train[maska]['index'].count()
    return new_train


def feature_quest_otvet(new_train, train, quest, kol_f):
    global kol_col
    kol_col = 9
    g1 = 0.7 
    g2 = 0.3 

    feature_q = feature_df[feature_df['quest'] == quest].copy()
    feature_q.reset_index(drop=True, inplace=True)
    
    gran1 = round(kol_f * g1)
    gran2 = round(kol_f * g2)    
    for i in range(0, kol_f):         
        row_f = feature_q.loc[i]
        new_train = feature_next_t_otvet(row_f, new_train, train, i < gran1, i <  gran2, i) 
    col = [i for i in range(0,kol_col+1)]
    return new_train[col]


def feature_engineer_new(new_train, train, feature_q, kol_f):
    g1 = 0.7 
    g2 = 0.3 
    gran1 = round(kol_f * g1)
    gran2 = round(kol_f * g2)    
    for i in range(0, kol_f): 
        row_f = feature_q.loc[i]       
        new_train = feature_next_t(row_f, new_train, train, i < gran1, i <  gran2, i)         
    return new_train


def feature_quest(new_train, train, quest, kol_f):
    global kol_col
    kol_col = 9
    feature_q = feature_df[feature_df['quest'] == quest].copy()
    feature_q.reset_index(drop=True, inplace=True)
    new_train = feature_engineer_new(new_train, train, feature_q, kol_f)
    col = [i for i in range(0,kol_col+1)]
    return new_train[col]


def create_model(old_train, quests, models, list_kol_f):
    
    kol_quest = len(quests)
    # ITERATE THRU QUESTIONS
    for q in quests:
        print('### quest ', q, end='')
        new_train = feature_engineer(old_train, list_kol_f[q])
        train_x = feature_quest(new_train, old_train, q, list_kol_f[q])
        print (' ---- ', 'train_q.shape = ', train_x.shape)
           
        # TRAIN DATA
        train_users = train_x.index.values
        train_y = targets.loc[targets.q==q].set_index('session').loc[train_users]

        # TRAIN MODEL 

        model = CatBoostClassifier(
            n_estimators = 300,
            learning_rate= 0.045,
            depth = 6
        )
        
        model.fit(train_x.astype('float32'), train_y['correct'], verbose=False)

        # SAVE MODEL, PREDICT VALID OOF
        models[f'{q}'] = model
    print('***')
    
    return models

In [ ]:
df0_4 = pd.read_csv('/kaggle/input/feature-for-cat/train_0_4t.csv', dtype=dtypes) 
kol_lvl = (df0_4 .groupby(['session_id'])['level'].agg('nunique') < 5)
list_session = kol_lvl[kol_lvl].index
df0_4  = df0_4 [~df0_4 ['session_id'].isin(list_session)]
df0_4 = delt_time_def(df0_4)

quests_0_4 = [1, 3] 

models = create_model(df0_4, quests_0_4, models, list_kol_f)
del df0_4

In [ ]:
df5_12 = pd.read_csv('/kaggle/input/feature-for-cat/train_5_12t.csv', dtype=dtypes)
kol_lvl = (df5_12.groupby(['session_id'])['level'].agg('nunique') < 8)
list_session = kol_lvl[kol_lvl].index
df5_12 = df5_12[~df5_12['session_id'].isin(list_session)]
df5_12 = delt_time_def(df5_12)
quests_5_12 = [4, 5, 6, 7, 8, 9, 10, 11] 

models = create_model(df5_12, quests_5_12, models, list_kol_f)
del df5_12

In [ ]:
df13_22 = pd.read_csv('/kaggle/input/feature-for-cat/train_13_22t.csv', dtype=dtypes) 
kol_lvl = (df13_22 .groupby(['session_id'])['level'].agg('nunique') < 10)
list_session = kol_lvl[kol_lvl].index
df13_22  = df13_22 [~df13_22 ['session_id'].isin(list_session)]
df13_22 = delt_time_def(df13_22)

quests_13_22 = [14, 15, 16, 17] 

models = create_model(df13_22, quests_13_22, models, list_kol_f)

# XGboost Part

In [ ]:
LIST_CATS = ['event_name', 'name', 'fqid', 'room_fqid', 'text_fqid']
LIST_NUMS = ['page', 'room_coor_x', 'room_coor_y', 'screen_coor_x', 'screen_coor_y', 'hover_duration', 'elapsed_time_diff']

LIST_NAME_FEATURE = ['basic', 'undefined', 'close', 'open', 'prev', 'next']
LIST_EVENT_NAME_FEATURE = ['cutscene_click', 'person_click', 'navigate_click', 'observation_click', 'notification_click', 'object_click', 'object_hover', 'map_hover', 'map_click', 'checkpoint', 'notebook_click']

LIST_FQID = [
     'worker',
     'archivist',
     'gramps',
     'wells',
     'toentry',
     'confrontation',
     'crane_ranger',
     'groupconvo',
     'flag_girl',
     'tomap',
     'tostacks',
     'tobasement',
     'archivist_glasses',
     'boss',
     'journals',
     'seescratches',
     'groupconvo_flag',
     'cs',
     'teddy',
     'expert',
     'businesscards',
     'ch3start',
     'tunic.historicalsociety',
     'tofrontdesk',
     'savedteddy',
     'plaque',
     'glasses',
     'tunic.drycleaner',
     'reader_flag',
     'tunic.library',
     'tracks',
     'tunic.capitol_2',
     'trigger_scarf',
     'reader',
     'directory',
     'tunic.capitol_1',
     'journals.pic_0.next',
     'unlockdoor',
     'tunic',
     'what_happened',
     'tunic.kohlcenter',
     'tunic.humanecology',
     'colorbook',
     'logbook',
     'businesscards.card_0.next',
     'journals.hub.topics',
     'logbook.page.bingo',
     'journals.pic_1.next',
     'journals_flag',
     'reader.paper0.next',
     'tracks.hub.deer',
     'reader_flag.paper0.next',
     'trigger_coffee',
     'wellsbadge',
     'journals.pic_2.next',
     'tomicrofiche',
     'journals_flag.pic_0.bingo',
     'plaque.face.date',
     'notebook',
     'tocloset_dirty',
     'businesscards.card_bingo.bingo',
     'businesscards.card_1.next',
     'tunic.wildlife',
     'tunic.hub.slip',
     'tocage',
     'journals.pic_2.bingo',
     'tocollectionflag',
     'tocollection',
     'chap4_finale_c',
     'chap2_finale_c',
     'lockeddoor',
     'journals_flag.hub.topics',
     'tunic.capitol_0',
     'reader_flag.paper2.bingo',
     'photo',
     'tunic.flaghouse',
     'reader.paper1.next',
     'directory.closeup.archivist',
     'intro',
     'businesscards.card_bingo.next',
     'reader.paper2.bingo',
     'retirement_letter',
     'remove_cup',
     'journals_flag.pic_0.next',
     'magnify',
     'coffee',
     'key',
     'togrampa',
     'reader_flag.paper1.next',
     'janitor',
     'tohallway',
     'chap1_finale',
     'report',
     'outtolunch',
     'journals_flag.hub.topics_old',
     'journals_flag.pic_1.next',
     'reader.paper2.next',
     'chap1_finale_c',
     'reader_flag.paper2.next',
     'door_block_talk',
     'journals_flag.pic_1.bingo',
     'journals_flag.pic_2.next',
     'journals_flag.pic_2.bingo',
     'block_magnify',
     'reader.paper0.prev',
     'block',
     'reader_flag.paper0.prev',
     'block_0',
     'door_block_clean',
     'reader.paper2.prev',
     'reader.paper1.prev',
     'doorblock',
     'tocloset',
     'reader_flag.paper2.prev',
     'reader_flag.paper1.prev',
     'block_tomap2',
     'journals_flag.pic_0_old.next',
     'journals_flag.pic_1_old.next',
     'block_tocollection',
     'block_nelson',
     'journals_flag.pic_2_old.next',
     'block_tomap1',
     'block_badge',
     'need_glasses',
     'block_badge_2',
     'fox',
     'block_1'
]

LIST_TEXT = [
     'tunic.historicalsociety.cage.confrontation',
     'tunic.wildlife.center.crane_ranger.crane',
     'tunic.historicalsociety.frontdesk.archivist.newspaper',
     'tunic.historicalsociety.entry.groupconvo',
     'tunic.wildlife.center.wells.nodeer',
     'tunic.historicalsociety.frontdesk.archivist.have_glass',
     'tunic.drycleaner.frontdesk.worker.hub',
     'tunic.historicalsociety.closet_dirty.gramps.news',
     'tunic.humanecology.frontdesk.worker.intro',
     'tunic.historicalsociety.frontdesk.archivist_glasses.confrontation',
     'tunic.historicalsociety.basement.seescratches',
     'tunic.historicalsociety.collection.cs',
     'tunic.flaghouse.entry.flag_girl.hello',
     'tunic.historicalsociety.collection.gramps.found',
     'tunic.historicalsociety.basement.ch3start',
     'tunic.historicalsociety.entry.groupconvo_flag',
     'tunic.library.frontdesk.worker.hello',
     'tunic.library.frontdesk.worker.wells',
     'tunic.historicalsociety.collection_flag.gramps.flag',
     'tunic.historicalsociety.basement.savedteddy',
     'tunic.library.frontdesk.worker.nelson',
     'tunic.wildlife.center.expert.removed_cup',
     'tunic.library.frontdesk.worker.flag',
     'tunic.historicalsociety.frontdesk.archivist.hello',
     'tunic.historicalsociety.closet.gramps.intro_0_cs_0',
     'tunic.historicalsociety.entry.boss.flag',
     'tunic.flaghouse.entry.flag_girl.symbol',
     'tunic.historicalsociety.closet_dirty.trigger_scarf',
     'tunic.drycleaner.frontdesk.worker.done',
     'tunic.historicalsociety.closet_dirty.what_happened',
     'tunic.wildlife.center.wells.animals',
     'tunic.historicalsociety.closet.teddy.intro_0_cs_0',
     'tunic.historicalsociety.cage.glasses.afterteddy',
     'tunic.historicalsociety.cage.teddy.trapped',
     'tunic.historicalsociety.cage.unlockdoor',
     'tunic.historicalsociety.stacks.journals.pic_2.bingo',
     'tunic.historicalsociety.entry.wells.flag',
     'tunic.humanecology.frontdesk.worker.badger',
     'tunic.historicalsociety.stacks.journals_flag.pic_0.bingo',
     'tunic.historicalsociety.closet.intro',
     'tunic.historicalsociety.closet.retirement_letter.hub',
     'tunic.historicalsociety.entry.directory.closeup.archivist',
     'tunic.historicalsociety.collection.tunic.slip',
     'tunic.kohlcenter.halloffame.plaque.face.date',
     'tunic.historicalsociety.closet_dirty.trigger_coffee',
     'tunic.drycleaner.frontdesk.logbook.page.bingo',
     'tunic.library.microfiche.reader.paper2.bingo',
     'tunic.kohlcenter.halloffame.togrampa',
     'tunic.capitol_2.hall.boss.haveyougotit',
     'tunic.wildlife.center.wells.nodeer_recap',
     'tunic.historicalsociety.cage.glasses.beforeteddy',
     'tunic.historicalsociety.closet_dirty.gramps.helpclean',
     'tunic.wildlife.center.expert.recap',
     'tunic.historicalsociety.frontdesk.archivist.have_glass_recap',
     'tunic.historicalsociety.stacks.journals_flag.pic_1.bingo',
     'tunic.historicalsociety.cage.lockeddoor',
     'tunic.historicalsociety.stacks.journals_flag.pic_2.bingo',
     'tunic.historicalsociety.collection.gramps.lost',
     'tunic.historicalsociety.closet.notebook',
     'tunic.historicalsociety.frontdesk.magnify',
     'tunic.humanecology.frontdesk.businesscards.card_bingo.bingo',
     'tunic.wildlife.center.remove_cup',
     'tunic.library.frontdesk.wellsbadge.hub',
     'tunic.wildlife.center.tracks.hub.deer',
     'tunic.historicalsociety.frontdesk.key',
     'tunic.library.microfiche.reader_flag.paper2.bingo',
     'tunic.flaghouse.entry.colorbook',
     'tunic.wildlife.center.coffee',
     'tunic.capitol_1.hall.boss.haveyougotit',
     'tunic.historicalsociety.basement.janitor',
     'tunic.historicalsociety.collection_flag.gramps.recap',
     'tunic.wildlife.center.wells.animals2',
     'tunic.flaghouse.entry.flag_girl.symbol_recap',
     'tunic.historicalsociety.closet_dirty.photo',
     'tunic.historicalsociety.stacks.outtolunch',
     'tunic.library.frontdesk.worker.wells_recap',
     'tunic.historicalsociety.frontdesk.archivist_glasses.confrontation_recap',
     'tunic.capitol_0.hall.boss.talktogramps',
     'tunic.historicalsociety.closet.photo',
     'tunic.historicalsociety.collection.tunic',
     'tunic.historicalsociety.closet.teddy.intro_0_cs_5',
     'tunic.historicalsociety.closet_dirty.gramps.archivist',
     'tunic.historicalsociety.closet_dirty.door_block_talk',
     'tunic.historicalsociety.entry.boss.flag_recap',
     'tunic.historicalsociety.frontdesk.archivist.need_glass_0',
     'tunic.historicalsociety.entry.wells.talktogramps',
     'tunic.historicalsociety.frontdesk.block_magnify',
     'tunic.historicalsociety.frontdesk.archivist.foundtheodora',
     'tunic.historicalsociety.closet_dirty.gramps.nothing',
     'tunic.historicalsociety.closet_dirty.door_block_clean',
     'tunic.capitol_1.hall.boss.writeitup',
     'tunic.library.frontdesk.worker.nelson_recap',
     'tunic.library.frontdesk.worker.hello_short',
     'tunic.historicalsociety.stacks.block',
     'tunic.historicalsociety.frontdesk.archivist.need_glass_1',
     'tunic.historicalsociety.entry.boss.talktogramps',
     'tunic.historicalsociety.frontdesk.archivist.newspaper_recap',
     'tunic.historicalsociety.entry.wells.flag_recap',
     'tunic.drycleaner.frontdesk.worker.done2',
     'tunic.library.frontdesk.worker.flag_recap',
     'tunic.humanecology.frontdesk.block_0',
     'tunic.library.frontdesk.worker.preflag',
     'tunic.historicalsociety.basement.gramps.seeyalater',
     'tunic.flaghouse.entry.flag_girl.hello_recap',
     'tunic.historicalsociety.closet.doorblock',
     'tunic.drycleaner.frontdesk.worker.takealook',
     'tunic.historicalsociety.basement.gramps.whatdo',
     'tunic.library.frontdesk.worker.droppedbadge',
     'tunic.historicalsociety.entry.block_tomap2',
     'tunic.library.frontdesk.block_nelson',
     'tunic.library.microfiche.block_0',
     'tunic.historicalsociety.entry.block_tocollection',
     'tunic.historicalsociety.entry.block_tomap1',
     'tunic.historicalsociety.collection.gramps.look_0',
     'tunic.library.frontdesk.block_badge',
     'tunic.historicalsociety.cage.need_glasses',
     'tunic.library.frontdesk.block_badge_2',
     'tunic.kohlcenter.halloffame.block_0',
     'tunic.capitol_0.hall.chap1_finale_c',
     'tunic.capitol_1.hall.chap2_finale_c',
     'tunic.capitol_2.hall.chap4_finale_c',
     'tunic.wildlife.center.fox.concern',
     'tunic.drycleaner.frontdesk.block_0',
     'tunic.historicalsociety.entry.gramps.hub',
     'tunic.humanecology.frontdesk.block_1',
     'tunic.drycleaner.frontdesk.block_1'
]

LIST_ROOM = [
     'tunic.historicalsociety.entry',
     'tunic.wildlife.center',
     'tunic.historicalsociety.cage',
     'tunic.library.frontdesk',
     'tunic.historicalsociety.frontdesk',
     'tunic.historicalsociety.stacks',
     'tunic.historicalsociety.closet_dirty',
     'tunic.humanecology.frontdesk',
     'tunic.historicalsociety.basement',
     'tunic.kohlcenter.halloffame',
     'tunic.library.microfiche',
     'tunic.drycleaner.frontdesk',
     'tunic.historicalsociety.collection',
     'tunic.historicalsociety.closet',
     'tunic.flaghouse.entry',
     'tunic.historicalsociety.collection_flag',
     'tunic.capitol_1.hall',
     'tunic.capitol_0.hall',
     'tunic.capitol_2.hall'
]

In [ ]:
PATH_MODEL_PRE_RAW = '/kaggle/input/blend-3ways-feature/xgb_models/pre_raw/XGB_question{}.xgb'
PATH_FEATURE_PRE_RAW = '/kaggle/input/blend-3ways-feature/pre_raw_features.json'

DIALOGS = ['that', 'this', 'it', 'you','find','found','Found','notebook','Wells','wells','help','need', 'Oh','Ooh','Jo', 'flag', 'can','and','is','the','to']

LEVELS = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]

IS_USE_EXTRA = True

threshold = 0.625

In [ ]:
DICT_FEATURE_LIST = {}
with open(PATH_FEATURE_PRE_RAW, mode='r') as fp:
    dict_use_feature = json.load(fp)

DICT_FEATURE_LIST['pre_raw'] = dict_use_feature

In [ ]:
def preprocessing_pl(x):
    """preprocessiong raw data before feature enginnering."""
    columns = [
        pl.col("page").cast(pl.Float32),
        (
            (pl.col("elapsed_time").diff(1))
             .fill_null(0)
             .clip(0, 1e9)
             .over(["session_id", "level_group"])
             .alias("elapsed_time_diff")
        ),
        (
            (pl.col("screen_coor_x").diff(1))
             .abs()
             .over(["session_id", "level_group"])
            .alias("location_x_diff")
        ),
        (
            (pl.col("screen_coor_y").diff(1))
             .abs()
             .over(["session_id", "level_group"])
            .alias("location_y_diff")
        ),
        pl.col("fqid").fill_null("fqid_None"),
        pl.col("text_fqid").fill_null("text_fqid_None")
    ]

    x = (
          x.drop(["fullscreen", "hq", "music"])
          .with_columns(columns))
    return x


def feature_engineer_pl(x, grp, use_extra, feature_suffix):
    """create aggregate features for polars."""
    def except_handling(s):
        try:
            return s.max() - s.min()
        except:
            return 0

    aggs = [
        pl.col("index").count().alias(f"session_number_{feature_suffix}"),
        pl.col("text_fqid").filter(pl.col("text_fqid") == 'text_fqid_None').count().alias(f'null_count'),
        pl.col('fqid').filter(pl.col('fqid').str.starts_with('to')).count().alias('page_change_count'),
        pl.col('index').filter(pl.col('text').str.contains('?', literal=True)).count().alias('question_count'),
        pl.col('index').filter((pl.col("event_name") == 'observation_click') & (pl.col('text_fqid').str.contains('block'))).count().alias('block_count'),
        pl.col('index').filter((pl.col("event_name") == 'person_click') & (pl.col('text_fqid').str.contains('recap'))).count().alias('recap_count'),
        pl.col('index').filter((pl.col("event_name") == 'person_click') & (pl.col('text_fqid').str.contains('lost'))).count().alias('lost_count'),

        *[pl.col(c).drop_nulls().n_unique().alias(f"{c}_unique_{feature_suffix}") for c in LIST_CATS],
        *[pl.col('index').filter(pl.col('text').str.contains(c, literal=True)).count().alias(f'word_{c}') for c in DIALOGS],
        *[pl.col("elapsed_time_diff").filter((pl.col('text').str.contains(c, literal=True))).mean().alias(f'word_mean_{c}') for c in DIALOGS],
        *[pl.col("elapsed_time_diff").filter((pl.col('text').str.contains(c, literal=True))).std().alias(f'word_std_{c}') for c in DIALOGS],
        *[pl.col("elapsed_time_diff").filter((pl.col('text').str.contains(c, literal=True))).max().alias(f'word_max_{c}') for c in DIALOGS],
        *[pl.col("elapsed_time_diff").filter((pl.col('text').str.contains(c, literal=True))).sum().alias(f'word_sum_{c}') for c in DIALOGS],
        *[pl.col("elapsed_time_diff").filter((pl.col('text').str.contains(c, literal=True))).median().alias(f'word_median_{c}') for c in DIALOGS],

        *[pl.col(c).mean().alias(f"{c}_mean_{feature_suffix}") for c in LIST_NUMS],
        *[pl.col(c).median().alias(f"{c}_median_{feature_suffix}") for c in LIST_NUMS],
        *[pl.col(c).std().alias(f"{c}_std_{feature_suffix}") for c in LIST_NUMS],
        *[pl.col(c).min().alias(f"{c}_min_{feature_suffix}") for c in LIST_NUMS],
        *[pl.col(c).max().alias(f"{c}_max_{feature_suffix}") for c in LIST_NUMS],
        *[pl.col(c).sum().alias(f"{c}_sum_{feature_suffix}") for c in LIST_NUMS],

        *[pl.col("event_name").filter(pl.col("event_name") == c).count().alias(f"{c}_event_name_counts{feature_suffix}")for c in LIST_EVENT_NAME_FEATURE],
        *[pl.col("elapsed_time_diff").filter(pl.col("event_name")==c).mean().alias(f"{c}_ET_mean_{feature_suffix}") for c in LIST_EVENT_NAME_FEATURE],
        *[pl.col("elapsed_time_diff").filter(pl.col("event_name")==c).median().alias(f"{c}_ET_median_{feature_suffix}") for c in LIST_EVENT_NAME_FEATURE],
        *[pl.col("elapsed_time_diff").filter(pl.col("event_name")==c).std().alias(f"{c}_ET_std_{feature_suffix}") for c in LIST_EVENT_NAME_FEATURE],
        *[pl.col("elapsed_time_diff").filter(pl.col("event_name")==c).max().alias(f"{c}_ET_max_{feature_suffix}") for c in LIST_EVENT_NAME_FEATURE],

        *[pl.col("name").filter(pl.col("name") == c).count().alias(f"{c}_name_counts{feature_suffix}")for c in LIST_NAME_FEATURE],   
        *[pl.col("elapsed_time_diff").filter(pl.col("name")==c).mean().alias(f"{c}_ET_mean_{feature_suffix}") for c in LIST_NAME_FEATURE],
        *[pl.col("elapsed_time_diff").filter(pl.col("name")==c).median().alias(f"{c}_ET_median_{feature_suffix}") for c in LIST_NAME_FEATURE],
        *[pl.col("elapsed_time_diff").filter(pl.col("name")==c).max().alias(f"{c}_ET_max_{feature_suffix}") for c in LIST_NAME_FEATURE],
        *[pl.col("elapsed_time_diff").filter(pl.col("name")==c).std().alias(f"{c}_ET_std_{feature_suffix}") for c in LIST_NAME_FEATURE],
        *[pl.col("elapsed_time_diff").filter(pl.col("name")==c).sum().alias(f"{c}_ET_sum_{feature_suffix}") for c in LIST_NAME_FEATURE],

        *[pl.col("room_fqid").filter(pl.col("room_fqid") == c).count().alias(f"{c}_room_fqid_counts{feature_suffix}")for c in LIST_ROOM],
        *[pl.col("elapsed_time_diff").filter(pl.col("room_fqid") == c).std().alias(f"{c}_ET_std_{feature_suffix}") for c in LIST_ROOM],
        *[pl.col("elapsed_time_diff").filter(pl.col("room_fqid") == c).mean().alias(f"{c}_ET_mean_{feature_suffix}") for c in LIST_ROOM],
        *[pl.col("elapsed_time_diff").filter(pl.col("room_fqid") == c).median().alias(f"{c}_ET_median_{feature_suffix}") for c in LIST_ROOM],
        *[pl.col("elapsed_time_diff").filter(pl.col("room_fqid") == c).max().alias(f"{c}_ET_max_{feature_suffix}") for c in LIST_ROOM],
        *[pl.col("elapsed_time_diff").filter(pl.col("room_fqid") == c).sum().alias(f"{c}_ET_sum_{feature_suffix}") for c in LIST_ROOM],

        *[pl.col("fqid").filter(pl.col("fqid") == c).count().alias(f"{c}_fqid_counts{feature_suffix}")for c in LIST_FQID],
        *[pl.col("elapsed_time_diff").filter(pl.col("fqid") == c).std().alias(f"{c}_ET_std_{feature_suffix}") for c in LIST_FQID],
        *[pl.col("elapsed_time_diff").filter(pl.col("fqid") == c).mean().alias(f"{c}_ET_mean_{feature_suffix}") for c in LIST_FQID],
        *[pl.col("elapsed_time_diff").filter(pl.col("fqid") == c).median().alias(f"{c}_ET_median_{feature_suffix}") for c in LIST_FQID],
        *[pl.col("elapsed_time_diff").filter(pl.col("fqid") == c).max().alias(f"{c}_ET_max_{feature_suffix}") for c in LIST_FQID],
        *[pl.col("elapsed_time_diff").filter(pl.col("fqid") == c).sum().alias(f"{c}_ET_sum_{feature_suffix}") for c in LIST_FQID],

        *[pl.col("text_fqid").filter(pl.col("text_fqid") == c).count().alias(f"{c}_text_fqid_counts{feature_suffix}") for c in LIST_TEXT],
        *[pl.col("elapsed_time_diff").filter(pl.col("text_fqid") == c).std().alias(f"{c}_ET_std_{feature_suffix}") for c in LIST_TEXT],
        *[pl.col("elapsed_time_diff").filter(pl.col("text_fqid") == c).mean().alias(f"{c}_ET_mean_{feature_suffix}") for c in LIST_TEXT],
        *[pl.col("elapsed_time_diff").filter(pl.col("text_fqid") == c).median().alias(f"{c}_ET_median_{feature_suffix}") for c in LIST_TEXT],
        *[pl.col("elapsed_time_diff").filter(pl.col("text_fqid") == c).max().alias(f"{c}_ET_max_{feature_suffix}") for c in LIST_TEXT],
        *[pl.col("elapsed_time_diff").filter(pl.col("text_fqid") == c).sum().alias(f"{c}_ET_sum_{feature_suffix}") for c in LIST_TEXT],

        *[pl.col("location_x_diff").filter(pl.col("event_name") == c).mean().alias(f"{c}_ET_mean_x{feature_suffix}") for c in LIST_EVENT_NAME_FEATURE],
        *[pl.col("location_x_diff").filter(pl.col("event_name") == c).median().alias(f"{c}_ET_median_x{feature_suffix}") for c in LIST_EVENT_NAME_FEATURE],
        *[pl.col("location_x_diff").filter(pl.col("event_name") == c).std().alias(f"{c}_ET_std_x{feature_suffix}") for c in LIST_EVENT_NAME_FEATURE],
        *[pl.col("location_x_diff").filter(pl.col("event_name") == c).max().alias(f"{c}_ET_max_x{feature_suffix}") for c in LIST_EVENT_NAME_FEATURE],
        *[pl.col("location_x_diff").filter(pl.col("event_name") == c).min().alias(f"{c}_ET_min_x{feature_suffix}") for c in LIST_EVENT_NAME_FEATURE],

        *[pl.col("level").filter(pl.col("level") == c).count().alias(f"{c}_LEVEL_count{feature_suffix}") for c in LEVELS],
        *[pl.col("elapsed_time_diff").filter(pl.col("level") == c).std().alias(f"{c}_ET_std_{feature_suffix}") for c in LEVELS],
        *[pl.col("elapsed_time_diff").filter(pl.col("level") == c).mean().alias(f"{c}_ET_mean_{feature_suffix}") for c in LEVELS],
        *[pl.col("elapsed_time_diff").filter(pl.col("level") == c).sum().alias(f"{c}_ET_sum_{feature_suffix}") for c in LEVELS],
        *[pl.col("elapsed_time_diff").filter(pl.col("level") == c).median().alias(f"{c}_ET_median_{feature_suffix}") for c in LEVELS],
        *[pl.col("elapsed_time_diff").filter(pl.col("level") == c).max().alias(f"{c}_ET_max_{feature_suffix}") for c in LEVELS],
        ]

    df = x.groupby(["session_id"], maintain_order=True).agg(aggs).sort("session_id")

    dict_agg_feature_by_level = {
        '0-4':[
            pl.col("elapsed_time").filter(((pl.col('fqid') == 'tunic') & (pl.col('event_name') == 'navigate_click')) | (pl.col('text_fqid') == 'tunic.historicalsociety.collection.tunic.slip')).apply(except_handling).alias("slip_click_duration"),
            pl.col("index").filter(((pl.col('fqid') == 'tunic') & (pl.col('event_name') == 'navigate_click')) | (pl.col('text_fqid') == 'tunic.historicalsociety.collection.tunic.slip')).apply(except_handling).alias("slip_click_indexCount"),
            pl.col("elapsed_time").filter(((pl.col('fqid') == 'plaque') & (pl.col('event_name') == 'navigate_click')) | (pl.col('text_fqid') == 'tunic.kohlcenter.halloffame.plaque.face.date')).apply(except_handling).alias("shirt_era_search_duration"),
            pl.col("index").filter(((pl.col('fqid') == 'plaque') & (pl.col('event_name') == 'navigate_click')) | (pl.col('text_fqid') == 'tunic.kohlcenter.halloffame.plaque.face.date')).apply(except_handling).alias("shirt_era_search_indexCount"),
        ],
        '5-12':[
            pl.col("elapsed_time").filter((pl.col("text")=="Here's the log book.")|(pl.col("fqid")=='logbook.page.bingo')).apply(lambda s: s.max()-s.min()).alias("logbook_bingo_duration"),
            pl.col("index").filter((pl.col("text")=="Here's the log book.")|(pl.col("fqid")=='logbook.page.bingo')).apply(lambda s: s.max()-s.min()).alias("logbook_bingo_indexCount"),
            pl.col("elapsed_time").filter(((pl.col("event_name")=='navigate_click')&(pl.col("fqid")=='reader'))|(pl.col("fqid")=="reader.paper2.bingo")).apply(lambda s: s.max()-s.min()).alias("reader_bingo_duration"),
            pl.col("index").filter(((pl.col("event_name")=='navigate_click')&(pl.col("fqid")=='reader'))|(pl.col("fqid")=="reader.paper2.bingo")).apply(lambda s: s.max()-s.min()).alias("reader_bingo_indexCount"),
            pl.col("elapsed_time").filter(((pl.col("event_name")=='navigate_click')&(pl.col("fqid")=='journals'))|(pl.col("fqid")=="journals.pic_2.bingo")).apply(lambda s: s.max()-s.min()).alias("journals_bingo_duration"),
            pl.col("index").filter(((pl.col("event_name")=='navigate_click')&(pl.col("fqid")=='journals'))|(pl.col("fqid")=="journals.pic_2.bingo")).apply(lambda s: s.max()-s.min()).alias("journals_bingo_indexCount"),
            pl.col("elapsed_time").filter(((pl.col('fqid') == 'businesscards') & (pl.col('event_name') == 'navigate_click')) | (pl.col('text_fqid') == 'tunic.humanecology.frontdesk.businesscards.card_bingo.bingo')).apply(except_handling).alias("businesscard_bingo_duration"),
            pl.col("index").filter(((pl.col('fqid') == 'businesscards') & (pl.col('event_name') == 'navigate_click')) | (pl.col('text_fqid') == 'tunic.humanecology.frontdesk.businesscards.card_bingo.bingo')).apply(except_handling).alias("businesscard_bingo_indexCount"),

            # add 20230423
            pl.col("elapsed_time").filter(((pl.col('text_fqid') == "tunic.historicalsociety.frontdesk.archivist.need_glass_0")) | (pl.col('text_fqid') == "tunic.historicalsociety.frontdesk.magnify")).apply(except_handling).alias("search_grass_duration"),
            pl.col("index").filter(((pl.col('text_fqid') == "tunic.historicalsociety.frontdesk.archivist.need_glass_0")) | (pl.col('text_fqid') == "tunic.historicalsociety.frontdesk.magnify")).apply(except_handling).alias("search_grass_indexCount"),

            # add 20230506
            pl.col("elapsed_time").filter(pl.col('level_group') == '0-4').last().alias('level_1_last'),
            pl.col("elapsed_time").filter(pl.col('level_group') == '5-12').first().alias('level_2_first'),

            *[pl.col(c).filter(pl.col('level_group') == '5-12').mean().alias(f"{c}_mean_level2") for c in LIST_NUMS],
            *[pl.col(c).filter(pl.col('level_group') == '5-12').median().alias(f"{c}_median_level2") for c in LIST_NUMS],
            *[pl.col(c).filter(pl.col('level_group') == '5-12').std().alias(f"{c}_std_level2") for c in LIST_NUMS],
            *[pl.col(c).filter(pl.col('level_group') == '5-12').min().alias(f"{c}_min_level2") for c in LIST_NUMS],
            *[pl.col(c).filter(pl.col('level_group') == '5-12').max().alias(f"{c}_max_level2") for c in LIST_NUMS],
            *[pl.col(c).filter(pl.col('level_group') == '5-12').sum().alias(f"{c}_sum_level2") for c in LIST_NUMS],

        ],
        '13-22':[
            pl.col("elapsed_time").filter(((pl.col("event_name")=='navigate_click')&(pl.col("fqid")=='reader_flag'))|(pl.col("fqid")=="tunic.library.microfiche.reader_flag.paper2.bingo")).apply(lambda s: s.max()-s.min() if s.len()>0 else 0).alias("reader_flag_duration"),
            pl.col("index").filter(((pl.col("event_name")=='navigate_click')&(pl.col("fqid")=='reader_flag'))|(pl.col("fqid")=="tunic.library.microfiche.reader_flag.paper2.bingo")).apply(lambda s: s.max()-s.min() if s.len()>0 else 0).alias("reader_flag_indexCount"),
            pl.col("elapsed_time").filter(((pl.col("event_name")=='navigate_click')&(pl.col("fqid")=='journals_flag'))|(pl.col("fqid")=="journals_flag.pic_0.bingo")).apply(lambda s: s.max()-s.min() if s.len()>0 else 0).alias("journalsFlag_bingo_duration"),
            pl.col("index").filter(((pl.col("event_name")=='navigate_click')&(pl.col("fqid")=='journals_flag'))|(pl.col("fqid")=="journals_flag.pic_0.bingo")).apply(lambda s: s.max()-s.min() if s.len()>0 else 0).alias("journalsFlag_bingo_indexCount"),
            pl.col("elapsed_time").filter(((pl.col("event_name")=='navigate_click')&(pl.col("fqid")=='tracks'))|(pl.col("text")=="That hoofprint doesn't match the flag!")).apply(except_handling).alias("tracks_duration"),
            pl.col("index").filter(((pl.col("event_name")=='navigate_click')&(pl.col("fqid")=='tracks'))|(pl.col("text")=="That hoofprint doesn't match the flag!")).apply(except_handling).alias("tracks_indexCount"),

            # add 20230423
            pl.col("elapsed_time").filter(((pl.col('event_name') == "person_click") & (pl.col('text_fqid') == "tunic.historicalsociety.cage.teddy.trapped")) | ((pl.col('event_name') == "navigate_click") & (pl.col('fqid') == "unlockdoor"))).apply(except_handling).alias("search_key_duration"),
            pl.col("index").filter(((pl.col('event_name') == "person_click") & (pl.col('text_fqid') == "tunic.historicalsociety.cage.teddy.trapped")) | ((pl.col('event_name') == "navigate_click") & (pl.col('fqid') == "unlockdoor"))).apply(except_handling).alias("search_key_indexCount"),

            # add 20230506
            pl.col("elapsed_time").filter(pl.col('level_group') == '5-12').last().alias('level_2_last'),
            pl.col("elapsed_time").filter(pl.col('level_group') == '13-22').first().alias('level_3_first'),

            *[pl.col(c).filter(pl.col('level_group') == '13-22').mean().alias(f"{c}_mean_level3") for c in LIST_NUMS],
            *[pl.col(c).filter(pl.col('level_group') == '13-22').median().alias(f"{c}_median_level3") for c in LIST_NUMS],
            *[pl.col(c).filter(pl.col('level_group') == '13-22').std().alias(f"{c}_std_level3") for c in LIST_NUMS],
            *[pl.col(c).filter(pl.col('level_group') == '13-22').min().alias(f"{c}_min_level3") for c in LIST_NUMS],
            *[pl.col(c).filter(pl.col('level_group') == '13-22').max().alias(f"{c}_max_level3") for c in LIST_NUMS],
            *[pl.col(c).filter(pl.col('level_group') == '13-22').sum().alias(f"{c}_sum_level3") for c in LIST_NUMS],

        ]
    }

    if use_extra:
        if grp == '0-4':
            aggs = dict_agg_feature_by_level['0-4']
            tmp = x.groupby(["session_id"], maintain_order=True).agg(aggs).sort("session_id")
            df = df.join(tmp, on="session_id", how='left')

        if grp == '5-12':
            aggs = dict_agg_feature_by_level['0-4'] + dict_agg_feature_by_level['5-12']

            tmp = (x.groupby(["session_id"], maintain_order=True)
                    .agg(aggs)
                    .sort("session_id")
                    .with_columns(level1_answer_time=pl.col('level_2_first') - pl.col('level_1_last'))
                    .drop(['level_1_last', 'level_2_first']))

            df = df.join(tmp, on="session_id", how='left')

        if grp == '13-22':
            aggs = dict_agg_feature_by_level['0-4'] + dict_agg_feature_by_level['5-12'] + dict_agg_feature_by_level['13-22']

            tmp = (x.groupby(["session_id"], maintain_order=True)
                    .agg(aggs)
                    .sort("session_id")
                    .with_columns(level1_answer_time=pl.col('level_2_first') - pl.col('level_1_last'))
                    .with_columns(level2_answer_time=pl.col('level_3_first') - pl.col('level_2_last'))
                    .drop(['level_1_last', 'level_2_first', 'level_2_last', 'level_3_first']))

            df = df.join(tmp, on="session_id", how='left')

    return df.to_pandas()


def time_feature(df):
    df["year"] = df["session_id"].apply(lambda x: int(str(x)[:2])).astype(np.uint8)
    df["month"] = df["session_id"].apply(lambda x: int(str(x)[2:4])+1).astype(np.uint8)
    df["day"] = df["session_id"].apply(lambda x: int(str(x)[4:6])).astype(np.uint8)
    df["hour"] = df["session_id"].apply(lambda x: int(str(x)[6:8])).astype(np.uint8)
    df["minute"] = df["session_id"].apply(lambda x: int(str(x)[8:10])).astype(np.uint8)
    df["second"] = df["session_id"].apply(lambda x: int(str(x)[10:12])).astype(np.uint8)
    return df

# Inference Part

In [ ]:
import jo_wilder_310 as jo_wilder

try:
    jo_wilder.make_env.__called__ = False
    env.__called__ = False
    type(env)._state = type(type(env)._state).__dict__['INIT']
except:
    pass

env = jo_wilder.make_env()
iter_test = env.iter_test()

In [ ]:
limits = {'0-4':(1,4), '5-12':(4,14), '13-22':(14,19)}
list_q = {'0-4':quests_0_4, '5-12':quests_5_12, '13-22':quests_13_22}
dict_feature_suffix = {'0-4': 'grp1', '5-12': 'grp2', '13-22': 'grp3'}

dict_feature_pre_raw = {}
session_id = -9999

for (test, sample_submission) in iter_test:
    test = test.sort_values(by='index')  # NOTE: jo_wilder_310 API
    # sort question
    sample_submission['questions'] = sample_submission['session_id'].str.split('_q').apply(lambda x: int(x[1]))
    sample_submission = sample_submission.sort_values(by='questions', ascending=True).reset_index()
    sample_submission.loc[sample_submission.questions.isin([5, 8, 10, 13, 15]), 'correct'] = 0  
    sample_submission = sample_submission[['session_id', 'correct']]

    if session_id == test.session_id.values[0]:
        is_switch_id = True
    else:
        is_switch_id = False

    session_id = test.session_id.values[0]
    grp = test.level_group.values[0]
    a, b = limits[grp]
    old_train = delt_time_def(test[test.level_group == grp])

    # stacking用のデータフレームを初期化しておく
    if not is_switch_id:
        df_pred_pre_raw = pd.DataFrame({'session_id': [session_id]})

    feature_pre_raw = DICT_FEATURE_LIST['pre_raw'][grp].copy()

    # pre_rawモデル用のデータフレーム
    test = pl.from_pandas(test)
    test = preprocessing_pl(test)
    dict_feature_pre_raw[grp] = test

    # 特徴量作成(pre_raw)
    if grp == '5-12':
        test = pl.concat([dict_feature_pre_raw['0-4'], test])
    elif grp == '13-22':
        test = pl.concat([dict_feature_pre_raw['0-4'], dict_feature_pre_raw['5-12'], test])

    test = feature_engineer_pl(test, grp, use_extra=IS_USE_EXTRA, feature_suffix='')
    test = time_feature(test)
    test = test[['session_id'] + feature_pre_raw]

    for t in range(a, b):
        mask = sample_submission.session_id.str.endswith(f'q{t}')
        # CatBoost Part
        if t in list_q[grp]:
            new_train = feature_engineer(old_train, list_kol_f[t])
            new_train = feature_quest_otvet(new_train, old_train, t, list_kol_f[t])

            clf_cat = models[f'{t}']
            pred_cat = clf_cat.predict_proba(new_train.astype('float32'))[:,1]
        else:
            pred_cat = np.array([1])

        # XGBoost Part
        clf_xgb = XGBClassifier()
        clf_xgb.load_model(PATH_MODEL_PRE_RAW.format(t))

        if t == 1:
            df_feature_pre_raw = test.drop(columns=['session_id'])
        elif t > 1:
            df_feature_pre_raw = pd.merge(test, df_pred_pre_raw.iloc[:, :t], on='session_id', how='left').drop(columns=['session_id'])

        pred_xgb = clf_xgb.predict_proba(df_feature_pre_raw.astype('float32'))[:,1]
        df_pred_pre_raw[f'q_{t}'] = pred_xgb

        # blend
        p = (pred_cat * 0.3) + (pred_xgb * 0.7)

        sample_submission.loc[mask,'correct'] = np.where(p > threshold, int(1), int(0))

    env.predict(sample_submission)